In [1]:
import numpy as np

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

from nsopy import SGMDoubleSimpleAveraging as DSA
from nsopy.loggers import EnhancedDualMethodLogger
from smpspy.oracles import TwoStage_SMPS_InnerProblem

output_notebook()
%cd ..

C:\Users\vujanicr\PycharmProjects\smpspy_base


# Solving dual model using DSA with Entropy Prox Term

Instantiating inner problem

In [3]:
ip = TwoStage_SMPS_InnerProblem('./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10')

Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.sto ...
Stochastic model is of type SCENARIOS DISCRETE


First solving it with DSA

In [4]:
GAMMA = 1.0

dsa = DSA(ip.oracle, ip.projection_function, dimension=ip.dimension, gamma=GAMMA)
logger_dsa = EnhancedDualMethodLogger(dsa)

In [5]:
for iteration in range(1500):
    if not iteration%50:
        print('Iteration: {}, d_k={}'.format(iteration, dsa.d_k))
    dsa.dual_step()

Iteration: 0, d_k=-inf
Iteration: 50, d_k=-55.0159186353
Iteration: 100, d_k=-54.9786759617
Iteration: 150, d_k=-54.8958271191
Iteration: 200, d_k=-54.946123664
Iteration: 250, d_k=-54.9126206845
Iteration: 300, d_k=-54.8681531927
Iteration: 350, d_k=-54.8759044684
Iteration: 400, d_k=-54.83683095
Iteration: 450, d_k=-54.8435540524
Iteration: 500, d_k=-54.833795125
Iteration: 550, d_k=-54.8560302563
Iteration: 600, d_k=-54.8279813457
Iteration: 650, d_k=-54.8655052529
Iteration: 700, d_k=-54.8348583158
Iteration: 750, d_k=-54.8305845772
Iteration: 800, d_k=-54.8340387889
Iteration: 850, d_k=-54.8374382962
Iteration: 900, d_k=-54.8204466776
Iteration: 950, d_k=-54.8407756352
Iteration: 1000, d_k=-54.8468588731
Iteration: 1050, d_k=-54.8219773445
Iteration: 1100, d_k=-54.8287186107
Iteration: 1150, d_k=-54.8173967699
Iteration: 1200, d_k=-54.8214714153
Iteration: 1250, d_k=-54.8338867381
Iteration: 1300, d_k=-54.8255492724
Iteration: 1350, d_k=-54.8146888169
Iteration: 1400, d_k=-54.8334

Then get the required parameters (R is derived a posteriori)

In [53]:
R_a_posteriori = 2*np.linalg.norm(dsa.lambda_k, ord=1)
R_safe = R_a_posteriori
ip = TwoStage_SMPS_InnerProblem('./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10', R=R_safe)

print('A-posteriori R={}'.format(R_a_posteriori))

Parsing nominal model information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.cor and .tim ...
Parsing stochastic information from ./smpspy/benchmark_problems/2_caroe_schultz/caroe_schultz_10.sto ...
Stochastic model is of type SCENARIOS DISCRETE
A-posteriori R=11.3342614708


Solve it using DSA with Entropy prox function. **Note that the only difference is that we pass in softmax projection function!**

In [54]:
dsa_entropy = DSA(ip.oracle, ip.softmax_projection, dimension=ip.dimension, gamma=GAMMA)
logger_dsa_entropy = EnhancedDualMethodLogger(dsa_entropy)

In [55]:
for iteration in range(1500):
    if not iteration%50:
        print('Iteration: {}, d_k={}'.format(iteration, dsa_entropy.d_k))
    dsa_entropy.dual_step()

Iteration: 0, d_k=-inf
Iteration: 50, d_k=-104.664775043
Iteration: 100, d_k=-77.6032850181
Iteration: 150, d_k=-64.8006837128
Iteration: 200, d_k=-69.4366207999
Iteration: 250, d_k=-68.8870870704
Iteration: 300, d_k=-69.4836031969
Iteration: 350, d_k=-62.8941010181
Iteration: 400, d_k=-65.3668548187
Iteration: 450, d_k=-61.9954427092
Iteration: 500, d_k=-59.8977152359
Iteration: 550, d_k=-61.2207819641
Iteration: 600, d_k=-63.302004345
Iteration: 650, d_k=-57.7549417917
Iteration: 700, d_k=-62.3360596363
Iteration: 750, d_k=-57.8880403384
Iteration: 800, d_k=-58.5907520617
Iteration: 850, d_k=-59.2018565437
Iteration: 900, d_k=-58.0431947093
Iteration: 950, d_k=-59.3052305592
Iteration: 1000, d_k=-59.5699477609
Iteration: 1050, d_k=-59.6194218746
Iteration: 1100, d_k=-59.5486144569
Iteration: 1150, d_k=-57.0995397158
Iteration: 1200, d_k=-61.1700128899
Iteration: 1250, d_k=-57.430900397
Iteration: 1300, d_k=-58.6404067734
Iteration: 1350, d_k=-57.5096016578
Iteration: 1400, d_k=-60.00

In [56]:
logger_dsa.lambda_k_iterates[-1]

array([ 0.14400903, -0.24433813,  0.27291032, -0.20353668,  0.27981351,
       -0.40031441, -0.15550504,  0.99409693, -0.15900846, -0.09654058,
        0.148661  , -0.40031441, -0.19887074,  0.99409693, -0.17734202,
       -0.39408986, -0.15466759, -0.24905981])

In [57]:
logger_dsa_entropy.lambda_k_iterates[-1]

array([-0.35588056, -0.26087179,  0.45746355, -0.28390145,  0.11714875,
       -0.33479585, -0.36493897,  0.94124571, -0.31225491, -0.1651139 ,
        0.40498141, -0.33479585, -0.19357511,  0.81556548, -0.11701537,
       -0.55699758,  0.36407121,  0.17966524])

In [58]:
p = figure(title="comparison", x_axis_label='iteration', y_axis_label='d_k')

In [59]:
p.line(range(len(logger_dsa.d_k_iterates)), logger_dsa.d_k_iterates, legend="DSA, gamma={}".format(GAMMA, R_safe))
p.line(range(len(logger_dsa_entropy.d_k_iterates)), logger_dsa_entropy.d_k_iterates, legend="DSA Entropy, gamma={}, R={}".format(GAMMA, R_safe), color='red')

GlyphRenderer(id='aad7fece-6bc4-4ec6-a95c-9c36ab042d30', ...)

In [60]:
p.legend.location = "bottom_right"
show(p)